In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
WINDOW_MA = 10

Lectura de archivos

In [2]:
tweets_df = pd.read_csv("../data/tweets_with_sentiment_cryptobert.csv")
tweets_df['Created At'] = pd.to_datetime(tweets_df['Created At'], errors='coerce')

# profe tips: revisar, la siguiente linea, porque parece redundante, ya que la linea anterior, en si misma convierte
# la data en fecha
tweets_df['fecha'] = tweets_df['Created At'].dt.date

price_df = pd.read_csv("../data/datos_mercado_binance.csv")
price_df['fecha'] = pd.to_datetime(price_df['fecha'], errors='coerce')

In [3]:
price_df.head(5)

,simbolo,fecha,precio_apertura,precio_maximo,precio_minimo,precio_cierre,volumen
0,BTCUSDT,2023-01-01,16541.77,16628.00,16499.01,16616.75,96925.41374
1,BTCUSDT,2023-01-02,16617.17,16799.23,16548.70,16672.87,121888.57191
2,BTCUSDT,2023-01-03,16672.78,16778.40,16605.28,16675.18,159541.53733
3,BTCUSDT,2023-01-04,16675.65,16991.87,16652.66,16850.36,220362.18862
4,BTCUSDT,2023-01-05,16850.36,16879.82,16753.00,16831.85,163473.56641


In [4]:
tweets_df.head(5)

,Username,Text,Created At,Retweets,Likes,Sentiment,fecha
0,Peter McCormack 🏴‍☠️🇬🇧🇮🇪,Happy New Year to you all.\n\nI hope you all h...,2023-01-01 01:49:27+00:00,8.0,450.0,1,2023-01-01
1,Mohamed A. El-Erian,The notion of an exciting football game means ...,2023-01-01 01:55:27+00:00,3.0,419.0,1,2023-01-01
2,Jimmy Song (송재준),"10 years ago, I:\n\n* Bought more Bitcoin\n* S...",2023-01-01 06:48:00+00:00,21.0,531.0,1,2023-01-01
3,Jameson Lopp,Bitcoin average DAILY value change during:\n20...,2023-01-01 12:50:57+00:00,35.0,231.0,0,2023-01-01
4,Jameson Lopp,Bitcoin average DAILY value change during:\n20...,2023-01-01 12:50:57+00:00,35.0,231.0,0,2023-01-01


In [5]:
# We count by kind of feeling
conteo_sentimientos = tweets_df.groupby(['fecha', 'Sentiment']).size().unstack(fill_value=0)

# We recalculate totals and average
conteo_sentimientos['numero_noticias'] = conteo_sentimientos.sum(axis=1)
conteo_sentimientos['promedio_sentimiento'] = tweets_df.groupby('fecha')['Sentiment'].mean()

Transforma las columnas date en formato datetime.date

In [7]:
price_df['fecha'] = pd.to_datetime(price_df['fecha']).dt.date 
conteo_sentimientos = conteo_sentimientos.reset_index()  # si 'fecha' está en el índice
conteo_sentimientos['fecha'] = pd.to_datetime(conteo_sentimientos['fecha']).dt.date
conteo_sentimientos.rename(
    columns={
        -1:'negative_sentiment_news',
        0:'neutral_sentiment_news',
        1:'positive_sentiment_news'
    },
    inplace=True
)


In [8]:
conteo_sentimientos.head(5)


Sentiment,index,fecha,negative_sentiment_news,neutral_sentiment_news,positive_sentiment_news,numero_noticias,promedio_sentimiento
0,0,2023-01-01,0,10,5,15,0.333333
1,1,2023-01-02,0,2,0,2,0.000000
2,2,2023-01-03,0,5,3,8,0.375000
3,3,2023-01-04,2,4,5,11,0.272727
4,4,2023-01-05,1,6,6,13,0.384615


Realiza la fusión de los dos DataFrames (price_df y conteo_sentimientos) usando la columna fecha como clave



In [9]:
# We join by date
df_final = price_df.merge(conteo_sentimientos, on='fecha', how='left')

In [10]:
df_final.head(5)

,simbolo,fecha,precio_apertura,precio_maximo,precio_minimo,precio_cierre,volumen,index,negative_sentiment_news,neutral_sentiment_news,positive_sentiment_news,numero_noticias,promedio_sentimiento
0,BTCUSDT,2023-01-01,16541.77,16628.00,16499.01,16616.75,96925.41374,0.0,0.0,10.0,5.0,15.0,0.333333
1,BTCUSDT,2023-01-02,16617.17,16799.23,16548.70,16672.87,121888.57191,1.0,0.0,2.0,0.0,2.0,0.000000
2,BTCUSDT,2023-01-03,16672.78,16778.40,16605.28,16675.18,159541.53733,2.0,0.0,5.0,3.0,8.0,0.375000
3,BTCUSDT,2023-01-04,16675.65,16991.87,16652.66,16850.36,220362.18862,3.0,2.0,4.0,5.0,11.0,0.272727
4,BTCUSDT,2023-01-05,16850.36,16879.82,16753.00,16831.85,163473.56641,4.0,1.0,6.0,6.0,13.0,0.384615


CALCULO MEDIA MOVIL CON DERIVADA Y TENDENCIA

In [14]:

# We added trend column
df_final['media_movil_10d'] = df_final['precio_apertura'].rolling(WINDOW_MA).mean()

# PROFE TIPS: la derivada hecha con la funcion diff no es la mejor opcion
# revisar otras funciones de derivadas
df_final['derivada_mm10'] = df_final['media_movil_10d'].diff()
condition_list = [
    df_final['derivada_mm10'].isna(),
    df_final['derivada_mm10'] > 0,
    df_final['derivada_mm10'] < 0,
]
returns_values_list = [0, 1, -1]
df_final['tendencia_categorizada'] = np.select(condition_list, returns_values_list, 0)
#def clasificar_derivada(valor):
#    if pd.isna(valor):
#        return 0  
#    elif valor > 0:
#        return 1
#    elif valor < 0:
#        return -1
#    else:
#        return 0
#df_final['tendencia_categorizada'] = df_final['derivada_mm10'].apply(clasificar_derivada)


📈 Indicador RSI (Relative Strength Index)

📊 RSI toma valores entre 0 y 100:

- (<30) podría indicar sobreventa.
- (>70) podría indicar sobrecompra.

In [18]:
# PROFE TIPS: trata de no usar funciones creadas por ti para los indicadores

def calcular_rsi(series, window=10):
    delta = series.diff()
    ganancias = delta.clip(lower=0)
    perdidas = -delta.clip(upper=0)

    media_ganancia = ganancias.rolling(window).mean()
    media_perdida = perdidas.rolling(window).mean()

    rs = media_ganancia / media_perdida
    rsi = 100 - (100 / (1 + rs))
    return rsi

df_final['RSI_10'] = calcular_rsi(df_final['precio_apertura'])

🔁 Indicador MACD (Moving Average Convergence Divergence)


In [19]:
# EMA rápida y lenta
ema_12 = df_final['precio_apertura'].ewm(span=12, adjust=False).mean()
ema_26 = df_final['precio_apertura'].ewm(span=26, adjust=False).mean()

# Línea MACD y señal
df_final['MACD'] = ema_12 - ema_26
df_final['MACD_signal'] = df_final['MACD'].ewm(span=9, adjust=False).mean()

In [20]:
# Selecting final columns
df_csv = df_final[['fecha', 'precio_apertura','media_movil_10d', 'RSI_10', 'MACD_signal', 'tendencia_categorizada', 'numero_noticias', -1, 0, 1, 'promedio_sentimiento']]

# We rename columns
df_csv.columns = ['DATE', 'OPEN','MA','RSI','MACD', 'TREND', 'NEWS ISSUE', 'NEGATIVE', 'NEUTRAL', 'POSITIVE', 'AVERAGE NEWS']

# We save CSV
df_csv.to_csv('../data/analisis_completo.csv', index=False)

KeyError: '[-1, 0, 1] not in index'